In [6]:
import numpy as np
from numpy.linalg import det, inv, eigvals, norm, matrix_rank
import scipy as sp
from scipy.linalg import null_space, sqrtm

np.set_printoptions(precision=2)

Real finite collection of vectors:<br>
$$F=(f_i)_{i\in I}\subset \mathbb{R}^n$$
$$I=\{1,...,m\}$$
$$m/n\; \dots \text{ "redundancy"}$$
$F$ is a frame if there are $A,B>0$ such that $$A\cdot \|f\|^2 \leq \sum_{i\in I} \vert \langle f,f_i \rangle \vert^2 \leq B\cdot \|f\|^2$$
for all $f\in \mathbb{R}^n$. This is equivalent to $F$ being a spanning set for $\mathbb{R}^n$ (we'll see) and $A,B$ indicate the numerical stability (we'll see). Clearly, $ m\geq n$ has to hold. <br>

Today: Real random frames:<br>
$$f_i \sim \mathcal{N}(0,1_n)\; \text{i.i.d. for all } i\in I$$

In [7]:
# dimensions
m = 6
n = 3

Analysis operator maps a vector $f\in \mathbb{R}^n$ to its frame coefficients via inner products with the $f_i$.
$\begin{align}
    T^*:\mathbb{R}^n &\rightarrow \mathbb{R}^m \\ f&\mapsto\left(\langle f, f_i\rangle\right)_{i\in I} = \left(\langle f_i, f\rangle\right)_{i\in I}
\end{align}$
Matrix multiplication from the left:
$$
T^* f = \begin{pmatrix}
    -\quad  f_1\quad  -\\
  
    -\quad  f_2\quad  -\\
      \vdots  \\
    -\quad  f_m\quad  -
\end{pmatrix} f
$$
$F$ is a frame iff $T^*$ is injective iff $T^*$ has full (column) rank iff there is $A>0$ such that $A\cdot \Vert f \Vert^2 \leq \Vert T^* f\Vert^2 = \sum_{i\in I} \vert \langle f,f_i \rangle \vert^2$.

Random setting: $T^*$ is a Gaussian matrix, which has full rank with probability one, i.e., a random collections of random vectors is a frame with prob one.

In [8]:
T_ana = np.random.randn(m, n)
print('T* =')
print(T_ana)
print('rank =', matrix_rank(T_ana))

f = np.random.randn(n)
Tf = T_ana @ f
print('frame coeff:', Tf)
# "change of basis"

T* =
[[ 0.2   1.15  1.03]
 [-0.72 -0.14 -1.55]
 [ 0.85 -0.18  0.23]
 [ 1.12  1.08  0.63]
 [-0.37 -1.16 -0.29]
 [ 0.48  0.69  0.34]]
rank = 3
frame coeff: [-0.68  0.27  0.33 -0.17  0.38 -0.18]


Synthesis operator maps frame coefficients $c = (c_i)_{i\in I}$ back to the signal space $\mathbb{R}^n$ via linear combinations of $f_i$.
\begin{align}
    T:\mathbb{R}^m &\rightarrow \mathbb{R}^n\\
    (c_i)_{i\in I}&\mapsto \sum_{i\in I} c_i\cdot f_i.
\end{align}
Matrix multiplication from the left:
$$T c = (T^*)^\top c = \begin{pmatrix}
    \vert & \vert & & \vert\\
    f_1 & f_2 & \cdots & f_m\\
    \vert & \vert & & \vert\\
\end{pmatrix} c$$
If $F$ is a frame, $T$ is surjective iff $T$ has full (row) rank.

In [9]:
T_syn = T_ana.T
print('T =')
print(T_syn)
print('rank =', matrix_rank(T_syn))

c = np.random.randn(m)
Tc = T_syn @ c
print('vector assoc with frame coefficients c:', Tc)

T =
[[ 0.2  -0.72  0.85  1.12 -0.37  0.48]
 [ 1.15 -0.14 -0.18  1.08 -1.16  0.69]
 [ 1.03 -1.55  0.23  0.63 -0.29  0.34]]
rank = 3
vector assoc with frame coefficients c: [-0.56  1.77  0.8 ]


Gramian does synthesis, followed by analysis.
\begin{align}
    G:\mathbb{R}^m &\rightarrow \mathbb{R}^m\\
    c&\mapsto \left(\left\langle \sum_{j\in I} c_j f_j,f_i \right\rangle\right)_{i\in I}.
\end{align}
As a matrix is carries the cross-correlations between the frame elements.
$$G=T^*T$$
$$G[i,j] = \langle f_i,f_j \rangle$$
$G$ is
- injective from $R_{T^*}$ to $R_{T^*}$ but not invertible on $\mathbb{R}^m$!
- self-adjoint
- the non-zero eigenvalues agree with those of $S$

In [10]:
G = T_ana @ T_syn
print('Gram =')
print(G)
print('det:',det(G))
print('self-adjoint:',np.all(G.T == G))

lam_G = eigvals(G)
#print(lam_G)
lam_pos = [l for l in lam_G if abs(l)>1e-10]
print('non-zero eigenvalues of G:')
print(np.real(lam_pos))
# whats the main diagonal of G?

Gram =
[[ 2.42 -1.89  0.19  2.11 -1.7   1.25]
 [-1.89  2.94 -0.93 -1.93  0.87 -0.97]
 [ 0.19 -0.93  0.8   0.89 -0.17  0.35]
 [ 2.11 -1.93  0.89  2.8  -1.84  1.49]
 [-1.7   0.87 -0.17 -1.84  1.56 -1.08]
 [ 1.25 -0.97  0.35  1.49 -1.08  0.83]]
det: -5.018735400409359e-47
self-adjoint: True
non-zero eigenvalues of G:
[8.67 0.94 1.74]


In [11]:
# the norms squared are on the diagonal
print(np.linalg.norm(T_ana[0,:])**2)

2.4216529364305206


Frame operator does analysis, followed by synthesis.
\begin{align}
    S:\mathbb{R}^n &\rightarrow \mathbb{R}^n\\
    f&\mapsto \sum_{i\in I} \langle f,f_i \rangle\cdot f_i.
\end{align}
As matrix:
$$S = TT^*$$
$S$ is
- invertible
- self-adjoint
- positive

In [12]:
S = T_syn @ T_ana
print('S =')
print(S)
print('determinant:',det(S))
print('self-adjoint:',np.all(S.T == S))
print('positive:',np.dot(S @ f,f))

lam = eigvals(S)
print('eigenvalues of S:')
print(lam)

S =
[[2.88 2.13 2.48]
 [2.13 4.36 2.6 ]
 [2.48 2.6  4.11]]
determinant: 14.113681209131995
self-adjoint: True
positive: 0.8508082979990015
eigenvalues of S:
[8.67 0.94 1.74]


The optimal frame bounds $A,B$ of $F$ are given by the smallest and largest eigenvalues of $S$.<br>
$B^{-1},A^{-1}$ are the smallest and largest eigenvalues of $S^{-1}$.

In [13]:
A = min(lam)
B = max(lam)
print('frame bounds:', A,B)

S_inv = inv(S)
lam_inv = eigvals(S_inv)
A_inv = min(lam_inv)
B_inv = max(lam_inv)
print('frame bounds of inverse:')
print(A_inv,B_inv)
print('inverse of frame bounds:')
print(B**(-1), A**(-1))

frame bounds: 0.9352063474251706 8.671660685300697
frame bounds of inverse:
0.11531816526159716 1.0692827339690538
inverse of frame bounds:
0.11531816526159708 1.0692827339690547


Canonical dual frame $\tilde{F}=(S^{-1}f_i)_{i\in I}$

In [14]:
# canonical dual frame
T_dual_syn = S_inv @ T_syn
print('synthesis operator:')
print(T_dual_syn)
print('check duality:')
S_mixed = T_dual_syn @ T_ana
print(S_mixed)
# this shows that every frame is dual to its dual
norm(f - S_inv @ T_syn @ T_ana @ f)

synthesis operator:
[[-0.42  0.03  0.61  0.47  0.    0.13]
 [ 0.27  0.31 -0.25  0.15 -0.36  0.15]
 [ 0.33 -0.59 -0.16 -0.23  0.16 -0.09]]
check duality:
[[ 1.00e+00  6.11e-17  1.73e-16]
 [ 1.52e-16  1.00e+00  6.41e-17]
 [ 1.44e-16 -1.74e-16  1.00e+00]]


2.4196749845665633e-16

framebounds of the canonical dual frame: $B^{-1},A^{-1}$

In [15]:
S_can = T_dual_syn @ T_dual_syn.T
lam_can = eigvals(S_can)
A_can = min(lam_can)
B_can = max(lam_can)
print(A_can, B_can)
print(B**(-1), A**(-1))

0.115318165261597 1.0692827339690532
0.11531816526159708 1.0692827339690547


Non-canonical dual frame: $\tilde{F}'=(S^{-1}f_i + \delta_i)_{i\in I}$ for $\delta_i \in \mathcal{N}_T$ (add elements form the null-space of the sythesis operator = range of the orthogonal complement of )

In [16]:
# ONB for the null-space of T_syn
B = null_space(T_syn)
f_kernel = B[:,0].reshape(1,m)
print('non-zero element in the kernel of F_syn:', f_kernel)
print('check:')
print(T_syn @ f_kernel.T)

non-zero element in the kernel of F_syn: [[-0.36 -0.17 -0.65  0.59  0.2  -0.18]]
check:
[[3.47e-16]
 [5.00e-16]
 [2.78e-16]]


In [17]:
T_dual2_syn = T_dual_syn + f_kernel
print('synthesis operator:')
print(T_dual2_syn)
print('check duality:')
print(T_dual2_syn @ T_ana)
# check that f_kernel is orthogonal to the range of T_ana
print('Since Tf_kern = 0, so is f_kernT*:')
print(f_kernel @ T_ana)

synthesis operator:
[[-0.78 -0.14 -0.03  1.06  0.2  -0.05]
 [-0.09  0.14 -0.89  0.74 -0.16 -0.03]
 [-0.03 -0.76 -0.81  0.36  0.36 -0.27]]
check duality:
[[1.00e+00 6.57e-16 5.50e-16]
 [4.34e-16 1.00e+00 3.60e-16]
 [4.49e-16 3.93e-16 1.00e+00]]
Since Tf_kern = 0, so is f_kernT*:
[[3.47e-16 5.00e-16 2.78e-16]]


Another non-canonical dual: sparse! choose a sub-frame of $F$ and compute the canonical dual for this one.

In [19]:
# choose 3 random frame elements
idx = np.random.randint(0, m, 3)
print(np.sort(idx))

[0 1 2]


In [20]:
S_small = T_syn[:,idx] @ T_ana[idx,:]
S_small_inv = inv(S_small)
T_dual3_small_syn = S_small_inv @ T_syn[:,idx]
T_dual3_syn = np.zeros((n,m))
T_dual3_syn[:,idx] = T_dual3_small_syn
print('synthesis operator:')
print(T_dual3_syn)
print('check duality:')
print(T_dual3_syn @ T_ana)

synthesis operator:
[[ 0.27  0.39  1.45  0.    0.    0.  ]
 [ 1.02  0.73  0.38  0.    0.    0.  ]
 [-0.22 -0.89 -0.71  0.    0.    0.  ]]
check duality:
[[ 1.00e+00 -5.09e-16 -2.15e-15]
 [-1.09e-16  1.00e+00 -7.36e-16]
 [ 3.10e-16  9.29e-17  1.00e+00]]


canonical dual has minimal $\ell^2$ norm

In [21]:
print('canonical:',norm(T_dual_syn))
print('non-canonical:',norm(T_dual2_syn))
print('non-canonical2:',norm(T_dual3_syn))

canonical: 1.3263506030526315
non-canonical: 2.1815604328594884
non-canonical2: 2.3195565407340446


Canonical tight frame $F_t=(S^{-1/2}f_i)_{i\in I}$

In [22]:
# canonical tight frame
S_inv_sqrt = sqrtm(S_inv)
T_syn_tight = S_inv_sqrt @ T_syn
T_ana_tight = T_syn_tight.T
S_tight = T_syn_tight @ T_ana_tight
print('frame operator:')
print(S_tight)

frame operator:
[[ 1.00e+00  1.62e-15  1.27e-15]
 [ 1.62e-15  1.00e+00 -5.17e-16]
 [ 1.27e-15 -5.17e-16  1.00e+00]]


Redundant frames, which ones can we remove?
\begin{align}
\langle f_i, S^{-1}f_i \rangle &\neq 1 \rightarrow \text{can remove it}\\
\langle f_i, S^{-1}f_i \rangle &= 1 \rightarrow \text{incomplete} 
\end{align}
intuition: if $\langle f_i, S^{-1}f_i \rangle = 1$, then $f_i$ spans its own 1-D sub-space

In [23]:
T_ana_2 = np.random.randn(n, n)
f_add = (np.random.randn(1)*T_ana_2[0,:] + np.random.randn(1)*T_ana_2[1,:]).reshape(1,n)
T_ana_dep = np.append(T_ana_2, f_add, axis=0)
print(T_ana_dep)

[[ 1.85  0.05  1.98]
 [ 0.05 -1.35 -0.73]
 [-2.18 -0.9  -1.06]
 [-2.23 -0.86 -2.85]]


In [24]:
S_dep = T_ana_dep.T @ T_ana_dep
S_dep_inv = inv(S_dep)
T_dual_syn_dep = S_dep_inv @ T_ana_dep.T
print('check condition:')
print(np.diag(T_ana_dep @ T_dual_syn_dep))
#[np.dot(T_ana_dep[k,:], T_dual_syn_dep[:,k]) for k in range(n+1)]

check condition:
[0.47 0.88 1.   0.65]
